In [ ]:
import pandas as pd
import os

In [ ]:
os.rename('./data/country_vaccinations.csv', './data/_raw_data.csv')
df = pd.read_csv('./data/_raw_data.csv')

In [ ]:
countries = list(set(df['country']))
dates = list(set(df['date']))
dates.sort()

In [ ]:
new_headers = ['date', 'daily_vaccinations', 'people_fully_vaccinated']
global_headers = ['country', 'date', 'daily_vaccinations', 'people_fully_vaccinated', 'vaccines']

In [ ]:
for i, country in enumerate(countries):
    country_df = df[df['country'] == country][new_headers]
    country_df.to_csv(f'./data/{country}_vaccinations.csv', index=False, header=True)

In [ ]:
data = []
for i, date in enumerate(dates):
    cur_dates_df = df[df['date']==date][new_headers]
    sum_column = list(cur_dates_df.sum(axis=0, numeric_only=True))
    sum_column.insert(0, date)
    data.append(sum_column)
dates_df = pd.DataFrame(data, columns=['date', 'daily_vaccinations', 'people_fully_vaccinated'])
dates_df.head

dates_df.to_csv('./data/Global_vaccinations.csv', index=False, header=True)